# Prediccion de ingresos laborales basado en la encuesta anual de hogares 2018

### Importando librerias


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Lectura del dataset

In [2]:
dataframe = pd.read_csv('./encuesta-anual-hogares-2018.csv', encoding='latin1')
dataframe.head(5)

,id,nhogar,miembro,comuna,dominio,edad,sexo,parentesco_jefe,situacion_conyugal,num_miembro_padre,...,ingreso_per_capita_familiar,estado_educativo,sector_educativo,nivel_actual,nivel_max_educativo,años_escolaridad,lugar_nacimiento,afiliacion_salud,hijos_nacidos_vivos,cantidad_hijos_nac_vivos
0,1,1,1,8,Resto de la Ciudad,16,Varon,Jefe,Soltero/a,Padre no vive en el hogar,...,2000,No asiste pero asistió,No corresponde,No corresponde,EGB (1° a 9° año),7,Pais limitrofe,Solo sistema publico,NaN,No corresponde
1,2,1,1,9,Villas de emergencia,18,Mujer,Jefe,Unido/a,Padre no vive en el hogar,...,5833,Asiste,Estatal/publico,Secundario/medio adultos,EGB (1° a 9° año),8,CABA,Solo sistema publico,Si,1
2,2,1,2,9,Villas de emergencia,37,Varon,Otro familiar,Unido/a,No corresponde,...,5833,No asiste pero asistió,No corresponde,No corresponde,Sala de 5,3,CABA,Solo sistema publico,NaN,No corresponde
3,2,1,3,9,Villas de emergencia,0,Varon,Hijo/a - Hijastro/a,No corresponde,Padre no vive en el hogar,...,5833,Nunca asistio,No corresponde,No corresponde,NaN,Ningun año de escolaridad aprobado,CABA,Solo sistema publico,NaN,No corresponde
4,3,1,1,2,Resto de la Ciudad,18,Mujer,Jefe,Soltero/a,Padre no vive en el hogar,...,6000,Asiste,Estatal/publico,Universitario,Otras escuelas especiales,12,PBA excepto GBA,Solo obra social,No,No corresponde


Se usa el metodo info para ver rapidamente que columnas tienen valores faltantes

In [3]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14497 entries, 0 to 14496
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           14497 non-null  int64 
 1   nhogar                       14497 non-null  int64 
 2   miembro                      14497 non-null  int64 
 3   comuna                       14497 non-null  int64 
 4   dominio                      14497 non-null  object
 5   edad                         14497 non-null  int64 
 6   sexo                         14497 non-null  object
 7   parentesco_jefe              14497 non-null  object
 8   situacion_conyugal           14496 non-null  object
 9   num_miembro_padre            14497 non-null  object
 10  num_miembro_madre            14497 non-null  object
 11  estado_ocupacional           14497 non-null  object
 12  cat_ocupacional              14497 non-null  object
 13  calidad_ingresos_lab         14

### Manejar datos faltantes

Se procede a hacer una funcion *columnas_nan* que guarda en una lista *columnas_val_faltantes* el nombre de la columna siempre y cuando tenga valores NaN, en caso de que ninguna columna tenga valores nulos devuelve string avisando que ya se termino de tratar con valores faltantes

In [4]:
def columnas_nan(df):
    columnas_val_faltantes = df.columns[df.isnull().any()].tolist()
    
    if columnas_val_faltantes:
        print(f"Columnas con NaN: {', '.join(columnas_val_faltantes)}")
    else:
        print("Ya no quedan valores faltantes")

In [5]:
columnas_nan(dataframe)

Columnas con NaN: situacion_conyugal, sector_educativo, nivel_max_educativo, años_escolaridad, lugar_nacimiento, afiliacion_salud, hijos_nacidos_vivos, cantidad_hijos_nac_vivos


#### hijos_nacidos_vivos
esta es la primer columna a completar, vemos que le faltan casi 8000 valores. Lo que se decidio hacer con esta columna fue compararla con cantidad de hijos nacidos vivos. Lo que ocurre es que se dejo el espacio vacio cuando la persona no tenia hijos vivos. Entonces la reemplazamos por *'No'*

In [6]:
dataframe['hijos_nacidos_vivos'].isnull().sum()

7978

In [10]:
dataframe[dataframe['hijos_nacidos_vivos'].isna()] ['cantidad_hijos_nac_vivos'].value_counts()

Series([], Name: count, dtype: int64)

In [8]:
dataframe['hijos_nacidos_vivos'] = dataframe['hijos_nacidos_vivos'].fillna('No')